In [1]:
from utils import GoogleRestaurantsDataset

dataset = GoogleRestaurantsDataset(fpath="/Users/jiaxi312/Desktop/INF385T/Final_Project/filter_all_t.json", train=True)
user_reviews, business_reviews, ratings = dataset.get_all_reviews()

In [5]:
import tensorflow as tf

from tensorflow import keras
from keras import layers
from models import TransformerEncoder, PositionalEmbedding, DeepFM

vocab_size = len(dataset.text_vectorize)
sequence_length = 1000
embed_dim = 64
num_heads = 8
dense_dim = 2048
num_feat = 128

dnn_layers = keras.Sequential(
    [PositionalEmbedding(sequence_length, vocab_size, embed_dim),
     TransformerEncoder(embed_dim, dense_dim, num_heads),
     layers.GlobalMaxPool1D(),
     layers.Dropout(0.2),
     layers.Dense(num_feat // 2, activation='relu')]
)

user_inputs = keras.Input(shape=(None,), dtype="int64")
user_outputs = dnn_layers(user_inputs)
business_inputs = keras.Input(shape=(None,), dtype="int64")
business_outputs = dnn_layers(business_inputs)
outputs = DeepFM(num_feat)(layers.concatenate([user_outputs, business_outputs], axis=1))

model = keras.Model([user_inputs, business_inputs], outputs)
model.compile(optimizer="rmsprop",
                  loss="mse",
                  metrics=["mean_squared_error"])

model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 sequential_7 (Sequential)      (None, 64)           2444800     ['input_5[0][0]',                
                                                                  'input_6[0][0]']                
                                                                                                  
 concatenate_2 (Concatenate)    (None, 128)          0           ['sequential_7[0][0]',     

In [6]:
model.fit([user_reviews, business_reviews], ratings)

2023-04-01 17:22:56.584759: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


  13/2720 [..............................] - ETA: 1:45:41 - loss: 21.1947 - mean_squared_error: 21.1947

KeyboardInterrupt: 